# 🚀 AI Banner Creator - Google Colab Edition
**Free GPU • Unlimited • Zero Cost • 5 min setup**

Tạo banner quảng cáo với AI Stable Diffusion (7GB) + GPU T4 MIỄN PHÍ

✓ GPU NVIDIA T4 (16GB) miễn phí  
✓ Tạo banner trong 30-60 giây  
✓ Lưu output về Google Drive  
✓ Không cần cài đặt gì  

In [ ]:
# Cell 1: Check GPU & Install Dependencies
!nvidia-smi

print('\n📦 Installing PyTorch with CUDA...')
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print('📦 Installing AI libraries...')
!pip install -q transformers diffusers accelerate pillow opencv-python rembg omegaconf einops

print('\n✅ GPU & Dependencies setup complete!')

In [ ]:
# Cell 2: Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Create directories
os.makedirs('/content/drive/My Drive/BannerCreator/input', exist_ok=True)
os.makedirs('/content/drive/My Drive/BannerCreator/output', exist_ok=True)

print('✅ Google Drive mounted!')
print('📁 Working directory: /content/drive/My Drive/BannerCreator/')

In [ ]:
# Cell 3: Download AI Models (First time: ~5-10 minutes)
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import torch

print('📥 Downloading Stable Diffusion 2.1 (7GB)...')
print('⏱️ This will take 3-5 minutes on first run')
print('⏳ Please wait...')

try:
    sd_pipeline = StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        torch_dtype=torch.float16,
        safety_checker=None
    )
    sd_pipeline = sd_pipeline.to("cuda")
    print('✅ Stable Diffusion 2.1 ready!')
except Exception as e:
    print(f'⚠️ Error: {e}')

print('\n✅ Models cached! Next runs will be instant.')

In [ ]:
# Cell 4: Define Banner Creator Class
import io
import json
from datetime import datetime
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import requests
from rembg import remove

class CoLabBannerCreator:
    def __init__(self):
        self.drive_path = "/content/drive/My Drive/BannerCreator"
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
    def download_sample_image(self):
        """Download sample product image"""
        sample_path = f"{self.drive_path}/input/sample_product.jpg"
        
        if not os.path.exists(sample_path):
            print('📥 Downloading sample product...')
            url = "https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=500"
            img = Image.open(io.BytesIO(requests.get(url).content))
            img.save(sample_path)
            print(f'✅ Saved to: {sample_path}')
        
        return sample_path
    
    def remove_background(self, image_path):
        """Remove background using rembg"""
        print(f'🎨 Removing background...')
        img = Image.open(image_path)
        img_no_bg = remove(img)
        return img_no_bg
    
    def generate_background(self, prompt, width=1000, height=600):
        """Generate background with Stable Diffusion"""
        print(f'🎨 Generating background...')
        
        with torch.no_grad():
            image = sd_pipeline(
                prompt=prompt,
                height=height,
                width=width,
                num_inference_steps=20,
                guidance_scale=7.5
            ).images[0]
        
        return image
    
    def create_simple_background(self, width, height):
        """Create simple gradient background"""
        bg = Image.new('RGB', (width, height))
        pixels = bg.load()
        
        for y in range(height):
            r = int(30 + (y / height) * 30)
            g = int(100 + (y / height) * 100)
            b = int(150 + (y / height) * 50)
            for x in range(width):
                pixels[x, y] = (r, g, b)
        
        return bg
    
    def create_banner(self, product_image_path, title, subtitle, bg_prompt=None, use_ai=True):
        """Create complete banner"""
        print(f'\n{"="*50}')
        print(f'📊 Creating Banner: {title}')
        print(f'{"="*50}')
        
        # Remove background
        product_img = self.remove_background(product_image_path)
        product_img.thumbnail((400, 400), Image.Resampling.LANCZOS)
        
        # Create or generate background
        if use_ai and bg_prompt:
            bg_img = self.generate_background(bg_prompt, width=1000, height=600)
        else:
            bg_img = self.create_simple_background(1000, 600)
        
        # Composite product
        if product_img.mode != 'RGBA':
            product_img = product_img.convert('RGBA')
        
        bg_img = bg_img.convert('RGBA')
        x = (bg_img.width - product_img.width) // 2
        y = (bg_img.height - product_img.height) // 2
        bg_img.paste(product_img, (x, y), product_img)
        bg_img = bg_img.convert('RGB')
        
        # Add text
        draw = ImageDraw.Draw(bg_img)
        try:
            title_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 48)
            subtitle_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 32)
        except:
            title_font = subtitle_font = ImageFont.load_default()
        
        draw.text((50, 50), title, font=title_font, fill=(255, 255, 255))
        draw.text((50, 120), subtitle, font=subtitle_font, fill=(200, 200, 200))
        
        # Save
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"{self.drive_path}/output/banner_{timestamp}.png"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        bg_img.save(output_path)
        
        print(f'✅ Banner saved!')
        return output_path

# Initialize
creator = CoLabBannerCreator()
print('✅ Banner Creator initialized!')

In [ ]:
# Cell 5: Create Single Banner with AI Background
# Download sample image
sample_path = creator.download_sample_image()

# Create banner
banner_path = creator.create_banner(
    product_image_path=sample_path,
    title="🎯 Premium Product",
    subtitle="Limited Time Offer - 50% OFF",
    bg_prompt="Modern office background with blue gradient, minimalist, professional",
    use_ai=True
)

print(f'\n🎉 Banner created!')
print(f'📁 File: {banner_path}')

In [ ]:
# Cell 6: Create Multiple Banners (Batch)
banners_config = [
    {
        "title": "🌞 Summer Sale",
        "subtitle": "Up to 70% OFF",
        "prompt": "bright sunny beach, golden sand, azure water, summer vibes, realistic"
    },
    {
        "title": "🎁 Black Friday",
        "subtitle": "Mega Deals",
        "prompt": "dark luxury background with red accents, premium feel, exclusive, dramatic"
    },
    {
        "title": "🚀 New Launch",
        "subtitle": "Be First to Own",
        "prompt": "futuristic technology background, neon lights, modern aesthetic, digital"
    }
]

print(f'📊 Creating {len(banners_config)} banners...\n')

for i, config in enumerate(banners_config, 1):
    print(f'[{i}/{len(banners_config)}]', end=" ")
    try:
        creator.create_banner(
            product_image_path=sample_path,
            title=config["title"],
            subtitle=config["subtitle"],
            bg_prompt=config["prompt"],
            use_ai=True
        )
    except Exception as e:
        print(f'❌ Error: {e}')

print(f'\n✅ All banners created!')

In [ ]:
# Cell 7: Display & Download Banners
import glob
from IPython.display import Image as IPImage, display

output_dir = creator.drive_path + "/output/"
banners = sorted(glob.glob(output_dir + "*.png"))

print(f'📸 Created {len(banners)} banners:\n')

# Display last 3 banners
for banner_path in banners[-3:]:
    print(f'\n📄 {os.path.basename(banner_path)}')
    display(IPImage(banner_path))

print(f'\n💾 All banners saved to Google Drive: {output_dir}')

In [ ]:
# Cell 8 (Optional): Create Banner from Your Own Image
# 1. Upload your image to Google Drive first
# 2. Update the path below
# 3. Run this cell

# Example: using your own product image
your_image_path = "/content/drive/My Drive/BannerCreator/input/your_product.jpg"

if os.path.exists(your_image_path):
    banner = creator.create_banner(
        product_image_path=your_image_path,
        title="🛍️ Your Custom Banner",
        subtitle="Create Now!",
        bg_prompt="Your custom background description here",
        use_ai=True
    )
    print(f'✅ Custom banner created: {banner}')
else:
    print(f'⚠️ Image not found: {your_image_path}')
    print('📝 How to use:')
    print('1. Upload image to Google Drive: BannerCreator/input/')
    print('2. Update path above')
    print('3. Run this cell again')

## 🎯 Bạn đã hoàn thành!

### ✅ Những gì bạn vừa làm:
1. ✓ Cài đặt PyTorch + AI libraries
2. ✓ Mount Google Drive
3. ✓ Download Stable Diffusion 2.1 (7GB)
4. ✓ Tạo Banner Creator class
5. ✓ Tạo banner đầu tiên
6. ✓ Batch create multiple banners
7. ✓ Display kết quả

### 📊 Hiệu năng:
- **GPU T4**: 30-60 giây/banner
- **CPU**: 2-5 phút/banner
- **Chi phí**: FREE (miễn phí)

### 🚀 Bước tiếp theo:
1. Upload ảnh sản phẩm của bạn
2. Chỉnh sửa prompt để tạo background phù hợp
3. Chạy Cell #8 để tạo banner từ ảnh của bạn
4. Download từ Google Drive

### 💡 Tips:
- Lần đầu: download models (5-10 phút)
- Lần sau: tức thì (models cached)
- GPU T4 có 12 giờ runtime → tạo được ~400 banners
- Muốn dùng lâu hơn? Subscribe Google One ($2/tháng)

Enjoy! 🎉